# LED IV Curve Fitting Tool

This tool analyzes LED current-voltage (IV) curves using a single diode equation ([Sandia model](https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.ivtools.sde.fit_sandia_simple.html#pvlib.ivtools.sde.fit_sandia_simple)) with a two-stage fitting approach focused on shunt and series resistances.


## How to Use

1. **Upload a CSV file** or select one from a directory using the dropdown
2. **Adjust the fitting ranges** if needed:
   - Sh2_R Range: For shunt resistance (typically negative voltage region)
   - S2_R Range: For series resistance (typically high voltage region)
3. **Click "Fit Current File"** to analyze data and view results
4. For multiple files, use **"Batch Fit All Files"**
5. **Save results** to CSV when finished

The tool will perform a two-stage fit:
- First determining Sh2_R in the specified range
- Then fixing Sh2_R and fitting other parameters focusing on S2_R

Results show fitted parameters (Sh2_R, S2_R, etc.) with quality metrics (R-squared and RMSE).

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
from scipy.optimize import curve_fit
import pvlib
from io import StringIO
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import glob
import json
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.dirname(os.getcwd()))
import access_token
access_token.log_notebook_usage()

class LEDFittingApp:
    def __init__(self):
        self.current_file = None
        self.data = None
        self.loaded_files = []
        self.fit_results = {}
        self.file_list = []
        
        # Initialize UI components
        self.create_ui()
        
    def create_ui(self):
        """Create the user interface elements"""
        # File selection
        self.file_upload = widgets.FileUpload(
            accept='.csv', 
            multiple=False,
            description='Upload File'
        )
        self.file_upload.observe(self.on_file_upload, names='value')
        
        # Directory path for batch processing
        self.batch_dir_input = widgets.Text(
            value='Raw_data',
            placeholder='Enter directory path for batch processing',
            description='Batch Directory:',
            disabled=False,
            layout=widgets.Layout(width='50%')
        )
        self.batch_dir_input.observe(self.on_dir_change, names='value')
        
        # File dropdown for directory files
        self.file_dropdown = widgets.Dropdown(
            options=[],
            description='Select File:',
            disabled=False,
            layout=widgets.Layout(width='50%')
        )
        self.file_dropdown.observe(self.on_file_dropdown_change, names='value')
        
        # Fitting range selectors
        self.shunt_range_selector = widgets.FloatRangeSlider(
            value=[-0.7, -0.5],
            min=-2.0,
            max=2.0,
            step=0.01,
            description='Sh2_R Range:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            layout=widgets.Layout(width='80%')
        )
        
        self.series_range_selector = widgets.FloatRangeSlider(
            value=[1.5, 2.0],
            min=-2.0,
            max=2.0,
            step=0.01,
            description='S2_R Range:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            layout=widgets.Layout(width='80%')
        )
        
        # Parameter sliders for initial values
        self.param_sliders = {
            'I_L': widgets.FloatLogSlider(
                value=1e-6,
                base=10,
                min=-9,  # 1e-9
                max=-3,  # 1e-3
                step=0.1,
                description='I_L (A):',
                readout_format='.2e',
                layout=widgets.Layout(width='80%')
            ),
            'I_o': widgets.FloatLogSlider(
                value=1e-12,
                base=10,
                min=-15,  # 1e-15
                max=-9,   # 1e-9
                step=0.1,
                description='I_o (A):',
                readout_format='.2e',
                layout=widgets.Layout(width='80%')
            ),
            'R_s': widgets.FloatLogSlider(
                value=1,
                base=10,
                min=-1,  # 0.1
                max=2,   # 100
                step=0.1,
                description='R_s (Ω):',
                readout_format='.2e',
                layout=widgets.Layout(width='80%')
            ),
            'n': widgets.FloatSlider(
                value=1.5,
                min=0.5,
                max=5.0,
                step=0.1,
                description='n (dim):',
                readout_format='.2f',
                layout=widgets.Layout(width='80%')
            ),
            'Sh2_R': widgets.FloatLogSlider(
                value=1e3,
                base=10,
                min=1,   # 10^1
                max=6,   # 10^6
                step=0.1,
                description='Sh2_R (Ω):',
                readout_format='.2e',
                layout=widgets.Layout(width='80%')
            ),
            'S2_R': widgets.FloatLogSlider(
                value=1e3,
                base=10,
                min=1,   # 10^1
                max=6,   # 10^6
                step=0.1,
                description='S2_R (Ω):',
                readout_format='.2e',
                layout=widgets.Layout(width='80%')
            )
        }
        
        # Action buttons
        self.scan_dir_button = widgets.Button(
            description='Scan Directory',
            button_style='info',
            tooltip='Scan directory for CSV files'
        )
        self.scan_dir_button.on_click(self.on_scan_dir_clicked)
        
        self.fit_button = widgets.Button(
            description='Fit Current File',
            button_style='primary',
            tooltip='Perform fitting on current file'
        )
        self.fit_button.on_click(self.on_fit_button_clicked)
        
        self.batch_fit_button = widgets.Button(
            description='Batch Fit All Files',
            button_style='success',
            tooltip='Fit all files in the specified directory'
        )
        self.batch_fit_button.on_click(self.on_batch_fit_clicked)
        
        self.save_results_button = widgets.Button(
            description='Save Results',
            button_style='info',
            tooltip='Save fitting results to a CSV file'
        )
        self.save_results_button.on_click(self.on_save_results_clicked)
        
        # Output areas
        self.plot_output = widgets.Output()
        self.results_output = widgets.Output()
        
        # Create tab layout
        tabs = widgets.Tab()
        tab_contents = [
            widgets.VBox([
                widgets.HBox([self.file_upload, self.batch_dir_input]),
                widgets.HBox([self.file_dropdown, self.scan_dir_button]),
                widgets.HTML("<h3>Fitting Range Selectors</h3>"),
                self.shunt_range_selector,
                self.series_range_selector,
                widgets.HTML("<h3>Initial Parameter Values</h3>"),
            ] + list(self.param_sliders.values()) + [
                widgets.HBox([self.fit_button, self.batch_fit_button, self.save_results_button]),
                self.plot_output
            ]),
            self.results_output
        ]
        tabs.children = tab_contents
        tabs.set_title(0, "Fitting Controls")
        tabs.set_title(1, "Results")
        
        # Display the UI
        display(HTML("<h1>LED IV Curve Fitting Tool</h1>"))
        display(tabs)
    
    def on_file_upload(self, change):
        """Handle file upload event"""
        if not change.new:
            return
            
        # Get the first uploaded file
        file_info = list(change.new.values())[0]
        content = file_info['content'].decode('utf-8')
        filename = file_info['name']
        
        with self.plot_output:
            self.plot_output.clear_output()
            print(f"Loading file: {filename}")
            
            # Parse the CSV file
            self.data = self.parse_csv(content)
            self.current_file = filename
            
            # Plot the data
            self.plot_data()
            
    def on_dir_change(self, change):
        """Handle directory input change event"""
        # This just updates the directory path but doesn't scan files
        # until the Scan Directory button is clicked
        pass
        
    def on_scan_dir_clicked(self, b):
        """Handle scan directory button click event"""
        directory = self.batch_dir_input.value
        
        with self.plot_output:
            self.plot_output.clear_output()
            
            if not os.path.exists(directory):
                print(f"Directory '{directory}' does not exist.")
                return
            
            # Find all CSV files in the directory
            csv_files = glob.glob(os.path.join(directory, "*.csv"))
            
            if not csv_files:
                print(f"No CSV files found in '{directory}'.")
                return
            
            print(f"Found {len(csv_files)} CSV files in '{directory}'.")
            
            # Update the dropdown with file options
            self.file_list = sorted(csv_files)
            self.file_dropdown.options = [os.path.basename(f) for f in self.file_list]
            
            if self.file_dropdown.options:
                self.file_dropdown.value = self.file_dropdown.options[0]
                
    def on_file_dropdown_change(self, change):
        """Handle file dropdown selection change"""
        if not change.new or not self.file_list:
            return
        
        # Find the full path from the selected filename
        selected_filename = change.new
        
        for full_path in self.file_list:
            if os.path.basename(full_path) == selected_filename:
                selected_file_path = full_path
                break
        else:
            return
        
        with self.plot_output:
            self.plot_output.clear_output()
            print(f"Loading file: {selected_filename}")
            
            try:
                # Read the file
                with open(selected_file_path, 'r') as f:
                    content = f.read()
                
                # Parse the CSV file
                self.data = self.parse_csv(content)
                self.current_file = selected_filename
                
                # Plot the data
                self.plot_data()
                
            except Exception as e:
                print(f"Error loading file: {str(e)}")
    
    def parse_csv(self, content):
        """Parse the CSV file content to extract data"""
        # Find the line index where the data starts
        lines = content.split('\n')
        data_start_line = 0
        for i, line in enumerate(lines):
            if line.startswith("Voltage (V),LED Current (A)"):
                data_start_line = i + 1
                break
        
        # Extract the data portion
        data_content = '\n'.join(lines[data_start_line:])
        
        # Parse using pandas
        df = pd.read_csv(StringIO(data_content), header=None)
        
        # Check if we have at least 2 columns
        if df.shape[1] >= 2:
            df.columns = ['Voltage', 'Current'] + [f'Col{i+3}' for i in range(df.shape[1]-2)]
            return df
        else:
            print("Error: Could not parse data properly.")
            return None
    
    def plot_data(self):
        """Plot the IV curve data"""
        if self.data is None:
            return
        
        fig = make_subplots(rows=1, cols=1)
        
        # Get data
        voltage = self.data['Voltage']
        current = self.data['Current'].abs()
        
        # Plot the data
        fig.add_trace(
            go.Scatter(
                x=voltage,
                y=current,
                mode='markers',
                name='Data',
                marker=dict(size=5, color='blue')
            )
        )
        
        # Set to log scale for y-axis
        fig.update_yaxes(type="log")
        
        # Auto-focus on the data
        y_min = current.min() * 0.5  # Add some padding below
        y_max = current.max() * 2    # Add some padding above
        
        # Update layout
        fig.update_layout(
            title=f"IV Curve - {self.current_file}",
            xaxis_title="Voltage (V)",
            yaxis_title="Current (A) - Log Scale",
            height=600,
            width=800,
            yaxis=dict(
                range=[np.log10(y_min), np.log10(y_max)]  # Set y-axis range in log scale
            ),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01
            )
        )
        
        fig.show()
    
    def sandia_simple_model(self, v, I_L, I_o, R_s, n, Sh2_R, S2_R):
        """
        Implement the Sandia simple model for LED IV curves
        This is based on pvlib's fit_sandia_simple but implemented directly for more control
        """
        k = 1.38066e-23  # Boltzmann's constant in J/K
        q = 1.60218e-19  # elementary charge in coulombs
        T = 298.15       # temperature in kelvin (25°C)
        thermal_voltage = k * T / q
        
        # Calculate the diode current
        v_diode = v - I_L * R_s  # Approximation for the first iteration
        
        # For numerical stability, limit exponential input
        exp_arg = np.minimum(v_diode / (n * thermal_voltage), 700)  # Avoid overflow
        I_diode = I_o * (np.exp(exp_arg) - 1)
        
        # Calculate currents through the shunt resistances
        I_sh2 = v_diode / Sh2_R
        I_s2 = v_diode / S2_R
        
        # Total current
        I = I_L - I_diode - I_sh2 - I_s2
        
        return I
    
    def fit_iv_curve(self, v, i, v_range=None, fixed_params=None):
        """
        Fit IV curve data using the Sandia simple model
        
        Parameters:
        -----------
        v : array
            Voltage data
        i : array
            Current data
        v_range : tuple
            Voltage range to use for the fit (min, max)
        fixed_params : dict
            Parameters to fix during the fit
            
        Returns:
        --------
        params : dict
            Fitted parameters
        pcov : array
            Covariance matrix
        """
        # Default initial parameter values
        initial_params = {
            'I_L': self.param_sliders['I_L'].value,
            'I_o': self.param_sliders['I_o'].value,
            'R_s': self.param_sliders['R_s'].value,
            'n': self.param_sliders['n'].value,
            'Sh2_R': self.param_sliders['Sh2_R'].value,
            'S2_R': self.param_sliders['S2_R'].value
        }
        
        # Filter data by voltage range if provided
        if v_range is not None:
            mask = (v >= v_range[0]) & (v <= v_range[1])
            v_fit = v[mask]
            i_fit = i[mask]
        else:
            v_fit = v
            i_fit = i
        
        # Determine which parameters to fit (exclude fixed ones)
        all_param_names = ['I_L', 'I_o', 'R_s', 'n', 'Sh2_R', 'S2_R']
        params_to_fit_names = [p for p in all_param_names if fixed_params is None or p not in fixed_params]
        
        # Define the model function with fixed parameters
        def model_func(x, *params_to_fit):
            # Create a complete parameter dictionary
            params_dict = {}
            
            # First add any fixed parameters
            if fixed_params:
                params_dict.update(fixed_params)
            
            # Then add the parameters that are being fitted
            for i, name in enumerate(params_to_fit_names):
                params_dict[name] = params_to_fit[i]
                
            # For any parameters that might be missing, use initial values
            for name in all_param_names:
                if name not in params_dict:
                    params_dict[name] = initial_params[name]
            
            # Call the model with the parameters
            return self.sandia_simple_model(
                x, 
                params_dict['I_L'],
                params_dict['I_o'],
                params_dict['R_s'],
                params_dict['n'],
                params_dict['Sh2_R'],
                params_dict['S2_R']
            )
        
        # Get initial values for parameters to fit
        p0 = [initial_params[name] for name in params_to_fit_names]
        
        try:
            # Perform the curve fitting
            popt, pcov = curve_fit(
                model_func, 
                v_fit, 
                i_fit, 
                p0=p0,
                maxfev=10000,
                method='lm'
            )
            
            # Create a dictionary of fitted parameters
            fitted_params = {}
            for i, name in enumerate(params_to_fit_names):
                fitted_params[name] = popt[i]
            
            # Add fixed parameters to the result
            if fixed_params:
                fitted_params.update(fixed_params)
            
            # Add any parameters that might be missing
            for name in all_param_names:
                if name not in fitted_params:
                    fitted_params[name] = initial_params[name]
            
            return fitted_params, pcov
            
        except Exception as e:
            print(f"Fitting error: {str(e)}")
            return initial_params, None
    
    def on_fit_button_clicked(self, b):
        """Handle fit button click event"""
        if self.data is None:
            with self.plot_output:
                print("No data loaded. Please upload a file first.")
            return
        
        with self.plot_output:
            self.plot_output.clear_output()
            print("Performing two-stage fitting...")
            
            # Get the voltage and current data
            voltage = self.data['Voltage'].values
            current = self.data['Current'].values
            
            # First fit: Focus on Sh2_R using the specified voltage range
            sh2_r_range = self.shunt_range_selector.value
            print(f"Stage 1: Fitting Sh2_R in voltage range: {sh2_r_range}")
            
            # For the first fit, we'll use specific parameters to focus on Sh2_R
            # We'll focus the fit on just the Sh2_R by only fitting this range
            first_fit_params, _ = self.fit_iv_curve(voltage, current, sh2_r_range)
            
            # Extract the fitted Sh2_R value from the first fit
            fitted_sh2_r = first_fit_params['Sh2_R']
            print(f"Stage 1 complete: Fitted Sh2_R = {fitted_sh2_r:.4e} Ω")
            
            # Second fit: Fix Sh2_R and focus on S2_R in the high voltage range
            fixed_params = {'Sh2_R': fitted_sh2_r}  # Fix the Sh2_R parameter
            
            # S2_R fitting range
            s2_r_range = self.series_range_selector.value
            print(f"Stage 2: Fitting with fixed Sh2_R = {fitted_sh2_r:.4e} Ω in voltage range: {s2_r_range}")
            
            # Perform the second fit with Sh2_R fixed
            final_params, pcov = self.fit_iv_curve(voltage, current, s2_r_range, fixed_params)
            
            print(f"Stage 2 complete: Fitted S2_R = {final_params['S2_R']:.4e} Ω")
            
            # Make sure Sh2_R is still the same (it should be since we fixed it)
            if abs(final_params['Sh2_R'] - fitted_sh2_r) > 1e-10:
                print("WARNING: Sh2_R value changed during the second fit! This should not happen.")
                print(f"Original Sh2_R: {fitted_sh2_r:.4e} Ω, Current Sh2_R: {final_params['Sh2_R']:.4e} Ω")
            
            # Calculate the full model prediction
            predicted_current = self.sandia_simple_model(
                voltage, 
                final_params['I_L'], 
                final_params['I_o'], 
                final_params['R_s'], 
                final_params['n'], 
                final_params['Sh2_R'], 
                final_params['S2_R']
            )
            
            # Calculate goodness of fit (R²)
            ss_tot = np.sum((current - np.mean(current))**2)
            ss_res = np.sum((current - predicted_current)**2)
            r_squared = 1 - (ss_res / ss_tot)
            
            # Calculate RMSE
            rmse = np.sqrt(np.mean((current - predicted_current)**2))
            
            # Store the results
            self.fit_results[self.current_file] = {
                'params': final_params,
                'r_squared': r_squared,
                'rmse': rmse
            }
            
            # Plot the results
            fig = make_subplots(rows=1, cols=1)
            
            # Plot the data
            fig.add_trace(
                go.Scatter(
                    x=voltage,
                    y=np.abs(current),
                    mode='markers',
                    name='Data',
                    marker=dict(size=5)
                )
            )
            
            # Get current data for y-axis limits
            abs_current = np.abs(current)
            y_min = abs_current.min() * 0.5  # Add padding below
            y_max = abs_current.max() * 2    # Add padding above
            
            # Plot the fit
            fig.add_trace(
                go.Scatter(
                    x=voltage,
                    y=np.abs(predicted_current),
                    mode='lines',
                    name='Complete Fit',
                    line=dict(color='red', width=2)
                )
            )
            
            # Highlight the Sh2_R fitting range
            x_sh2 = np.linspace(sh2_r_range[0], sh2_r_range[1], 100)
            y_sh2 = self.sandia_simple_model(
                x_sh2,
                final_params['I_L'], 
                final_params['I_o'], 
                final_params['R_s'], 
                final_params['n'], 
                final_params['Sh2_R'], 
                final_params['S2_R']
            )
            
            fig.add_trace(
                go.Scatter(
                    x=x_sh2,
                    y=np.abs(y_sh2),
                    mode='lines',
                    name='Sh2_R Range',
                    line=dict(color='green', width=4)
                )
            )
            
            # Highlight the S2_R fitting range
            x_s2 = np.linspace(s2_r_range[0], s2_r_range[1], 100)
            y_s2 = self.sandia_simple_model(
                x_s2,
                final_params['I_L'], 
                final_params['I_o'], 
                final_params['R_s'], 
                final_params['n'], 
                final_params['Sh2_R'], 
                final_params['S2_R']
            )
            
            fig.add_trace(
                go.Scatter(
                    x=x_s2,
                    y=np.abs(y_s2),
                    mode='lines',
                    name='S2_R Range',
                    line=dict(color='blue', width=4)
                )
            )
            
            # Set to log scale for y-axis
            fig.update_yaxes(type="log")
            
            # Update layout with focused y-range on the data
            fig.update_layout(
                title=f"IV Curve Fit - {self.current_file} (R² = {r_squared:.4f})",
                xaxis_title="Voltage (V)",
                yaxis_title="Current (A) - Log Scale",
                height=600,
                width=800,
                yaxis=dict(
                    range=[np.log10(y_min), np.log10(y_max)]  # Set y-axis range in log scale
                ),
                legend=dict(
                    yanchor="top",
                    y=0.99,
                    xanchor="left",
                    x=0.01
                )
            )
            
            fig.show()
            
            # Display the fit parameters
            print("\nFit Parameters:")
            for param, value in final_params.items():
                unit = ""
                if param in ['I_L', 'I_o']:
                    unit = "A"
                elif param in ['R_s', 'Sh2_R', 'S2_R']:
                    unit = "Ω"
                elif param == 'n':
                    unit = "(dimensionless)"
                    
                print(f"{param}: {value:.4e} {unit}")
            
            print(f"R²: {r_squared:.4f}")
            print(f"RMSE: {rmse:.4e} A")
            
            # Update the results tab
            with self.results_output:
                self.display_results()
    
    def on_batch_fit_clicked(self, b):
        """Handle batch fit button click event"""
        directory = self.batch_dir_input.value
        
        with self.plot_output:
            self.plot_output.clear_output()
            
            if not os.path.exists(directory):
                print(f"Directory '{directory}' does not exist.")
                return
            
            # Find all CSV files in the directory
            csv_files = glob.glob(os.path.join(directory, "*.csv"))
            
            if not csv_files:
                print(f"No CSV files found in '{directory}'.")
                return
            
            print(f"Found {len(csv_files)} CSV files. Starting batch processing...")
            
            # Process each file
            for i, file_path in enumerate(csv_files):
                filename = os.path.basename(file_path)
                print(f"Processing {i+1}/{len(csv_files)}: {filename}")
                
                try:
                    # Read the file
                    with open(file_path, 'r') as f:
                        content = f.read()
                    
                    # Parse the CSV file
                    data = self.parse_csv(content)
                    
                    if data is None:
                        print(f"  Failed to parse {filename}. Skipping.")
                        continue
                    
                    # Get the voltage and current data
                    voltage = data['Voltage'].values
                    current = data['Current'].values
                    
                    # First fit: Focus on Sh2_R
                    sh2_r_range = self.shunt_range_selector.value
                    print(f"  Stage 1: Fitting Sh2_R in voltage range: {sh2_r_range}")
                    first_fit_params, _ = self.fit_iv_curve(voltage, current, sh2_r_range)
                    
                    # Fix Sh2_R from the first fit
                    fitted_sh2_r = first_fit_params['Sh2_R']
                    fixed_params = {'Sh2_R': fitted_sh2_r}
                    
                    # Second fit: Focus on S2_R with fixed Sh2_R
                    s2_r_range = self.series_range_selector.value
                    print(f"  Stage 2: Fitting with fixed Sh2_R = {fitted_sh2_r:.4e} Ω in voltage range: {s2_r_range}")
                    final_params, pcov = self.fit_iv_curve(voltage, current, s2_r_range, fixed_params)
                    
                    # Check that Sh2_R remained fixed
                    if abs(final_params['Sh2_R'] - fitted_sh2_r) > 1e-10:
                        print(f"  WARNING: Sh2_R value changed during second fit from {fitted_sh2_r:.4e} to {final_params['Sh2_R']:.4e}")
                    
                    # Calculate the full model prediction
                    predicted_current = self.sandia_simple_model(
                        voltage, 
                        final_params['I_L'], 
                        final_params['I_o'], 
                        final_params['R_s'], 
                        final_params['n'], 
                        final_params['Sh2_R'], 
                        final_params['S2_R']
                    )
                    
                    # Calculate goodness of fit (R²)
                    ss_tot = np.sum((current - np.mean(current))**2)
                    ss_res = np.sum((current - predicted_current)**2)
                    r_squared = 1 - (ss_res / ss_tot)
                    
                    # Calculate RMSE
                    rmse = np.sqrt(np.mean((current - predicted_current)**2))
                    
                    # Store the results
                    self.fit_results[filename] = {
                        'params': final_params,
                        'r_squared': r_squared,
                        'rmse': rmse
                    }
                    
                    print(f"  Completed: R² = {r_squared:.4f}, Sh2_R = {final_params['Sh2_R']:.2e}, S2_R = {final_params['S2_R']:.2e}")
                    
                except Exception as e:
                    print(f"  Error processing {filename}: {str(e)}")
            
            print("\nBatch processing complete!")
            
            # Update the results tab
            with self.results_output:
                self.display_results()
    
    def on_save_results_clicked(self, b):
        """Handle save results button click event"""
        if not self.fit_results:
            with self.plot_output:
                print("No results to save. Please perform fitting first.")
            return
        
        with self.plot_output:
            try:
                # Create a DataFrame to store the results
                results_data = []
                
                for filename, result in self.fit_results.items():
                    row = {
                        'Filename': filename,
                        'R_squared': result['r_squared'],
                        'RMSE': result['rmse']
                    }
                    
                    # Add the parameters
                    for param, value in result['params'].items():
                        row[param] = value
                    
                    results_data.append(row)
                
                # Create the DataFrame
                results_df = pd.DataFrame(results_data)
                
                # Save to CSV
                output_file = "fitting_results.csv"
                results_df.to_csv(output_file, index=False)
                
                print(f"Results saved to {output_file}")
                
            except Exception as e:
                print(f"Error saving results: {str(e)}")
    
    def display_results(self):
        """Display the fitting results in a table"""
        self.results_output.clear_output()
        
        if not self.fit_results:
            print("No fitting results available.")
            return
        
        # Create a DataFrame to display the results
        results_data = []
        
        for filename, result in self.fit_results.items():
            row = {
                'Filename': filename,
                'Sh2_R (Ω)': result['params']['Sh2_R'],
                'S2_R (Ω)': result['params']['S2_R'],
                'I_L (A)': result['params']['I_L'],
                'I_o (A)': result['params']['I_o'],
                'R_s (Ω)': result['params']['R_s'],
                'n (dim)': result['params']['n'],
                'R²': result['r_squared'],
                'RMSE (A)': result['rmse']
            }
            
            results_data.append(row)
        
        # Create and display the DataFrame
        results_df = pd.DataFrame(results_data)
        
        # Format the values
        for param in ['Sh2_R (Ω)', 'S2_R (Ω)', 'I_L (A)', 'I_o (A)', 'R_s (Ω)', 'RMSE (A)']:
            results_df[param] = results_df[param].apply(lambda x: f"{x:.2e}")
        
        results_df['n (dim)'] = results_df['n (dim)'].apply(lambda x: f"{x:.2f}")
        results_df['R²'] = results_df['R²'].apply(lambda x: f"{x:.4f}")
        
        # Display the DataFrame as HTML
        display(HTML("<h2>Fitting Results Summary</h2>"))
        display(results_df)

# Create a Voila app
def run_app():
    app = LEDFittingApp()
    return app

# Run the app when the notebook is executed
if __name__ == "__main__":
    app = run_app()